# Transformer Model

In [ ]:
from tqdm import tqdm
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [ ]:
# Constants
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/MediaPipe_processed'
NUM_FRAMES = 90
save_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/final_DataSet'

# Label map
actions = sorted(os.listdir(DATA_PATH))
label_map = {label: idx for idx, label in enumerate(actions)}

def process_file(file_path):
    try:
        # Load .npy file
        sequence = np.load(file_path)

        # Normalize keypoints
        sequence = sequence / np.max(np.abs(sequence), axis=(0, 1), keepdims=True)

        # Pad or truncate to NUM_FRAMES
        return sequence[:NUM_FRAMES] if len(sequence) > NUM_FRAMES else np.pad(
            sequence, ((0, NUM_FRAMES - len(sequence)), (0, 0)), 'constant'
        )
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None  # Skip this file

# Count total files and prepare paths
all_files = [
    os.path.join(DATA_PATH, action, file)
    for action in os.listdir(DATA_PATH)
    for file in os.listdir(os.path.join(DATA_PATH, action))
    if file.endswith('.npy')
]

# Process files in parallel
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_file, all_files), total=len(all_files), desc="Processing Files"))

# Filter out None results
sequences = [seq for seq in results if seq is not None]
labels = [label_map[os.path.basename(os.path.dirname(file))] for file, seq in zip(all_files, results) if seq is not None]

# Convert to NumPy arrays
X = np.array(sequences)  # Shape: (num_samples, NUM_FRAMES, num_features)
y = to_categorical(labels, num_classes=len(label_map))  # Shape: (num_samples, num_classes)

# Create directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save X and y arrays
np.save(os.path.join(save_path, 'X.npy'), X)
np.save(os.path.join(save_path, 'y.npy'), y)

print("X and y saved successfully!")


Processing Files:  25%|██▌       | 3051/11980 [01:52<06:48, 21.86it/s]

Error processing file /content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/MediaPipe_processed/liability/33002_keypoints.npy: axis 1 is out of bounds for array of dimension 1


Processing Files: 100%|██████████| 11980/11980 [06:25<00:00, 31.07it/s] 


X and y saved successfully!


In [ ]:
# Load processed data from disk
save_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/final_DataSet'

X = np.load(os.path.join(save_path, 'X.npy'))
y = np.load(os.path.join(save_path, 'y.npy'))

In [ ]:
# Train-test-validation split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (9583, 90, 1662), y_train shape: (9583, 2000)
X_val shape: (1198, 90, 1662), y_val shape: (1198, 2000)
X_test shape: (1198, 90, 1662), y_test shape: (1198, 2000)


## AI Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
def transformer_block(inputs, num_heads, ff_dim, dropout=0.1):
    # Multi-head self-attention
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = LayerNormalization(epsilon=1e-6)(attention + inputs)

    # Feed-forward network
    ff = Dense(ff_dim, activation="relu")(attention)
    ff = Dense(inputs.shape[-1])(ff)
    ff = Dropout(dropout)(ff)
    outputs = LayerNormalization(epsilon=1e-6)(ff + attention)

    return outputs


In [ ]:
def build_transformer_model(seq_len, num_features, num_classes, num_heads=4, ff_dim=128, num_blocks=3, dropout=0.1):
    inputs = Input(shape=(seq_len, num_features))
    x = inputs

    # Add multiple Transformer blocks
    for _ in range(num_blocks):
        x = transformer_block(x, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout)

    # Global pooling and output layer
    x = GlobalAveragePooling1D()(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    return Model(inputs, outputs)


In [ ]:
# Model parameters
seq_len = 90  # Number of frames
num_features = 1662  # Features per frame
num_classes = 2000  # Number of classes
num_heads = 4
ff_dim = 128
num_blocks = 3

# Build the model
transformer_model = build_transformer_model(seq_len, num_features, num_classes, num_heads, ff_dim, num_blocks)
transformer_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])
transformer_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 90, 1662)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 90, 1662)       │     44,217,510 │ input_layer[0][0],     │
│ (MultiHeadAttention)      │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 90, 1662)       │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 90, 1662)       │              0 │ dropout_1[0][0],       │
│                           │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 90, 1662)       │          3,324 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 90, 128)        │        212,864 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 90, 1662)       │        214,398 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 90, 1662)       │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 90, 1662)       │              0 │ dropout_2[0][0],       │
│                           │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 90, 1662)       │          3,324 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 90, 1662)       │     44,217,510 │ layer_normalization_1… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 90, 1662)       │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 90, 1662)       │              0 │ dropout_4[0][0],       │
│                           │                        │                │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 90, 1662)       │          3,324 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 90, 128)        │        212,864 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)      

 Total params: 135,831,716 (518.16 MB)

 Trainable params: 135,831,716 (518.16 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Constants
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/MediaPipe_processed'
NUM_FRAMES = 90
save_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/final_DataSet'

# Label map
actions = sorted(os.listdir(DATA_PATH))
label_map = {label: idx for idx, label in enumerate(actions)}

# Define the checkpoint path
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/transformer_best_model.keras'

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss', verbose=1)

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(len(label_map)),
    y=np.argmax(y, axis=1)
)

# Convert to dictionary format for Keras
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# Train the model
history = transformer_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    class_weight=class_weight_dict,  # Add class weights here
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - categorical_accuracy: 4.7912e-04 - loss: 7.5812
Epoch 1: val_loss improved from inf to 7.61534, saving model to /content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/transformer_best_model.keras
300/300 ━━━━━━━━━━━━━━━━━━━━ 86s 237ms/step - categorical_accuracy: 4.8099e-04 - loss: 7.5812 - val_categorical_accuracy: 0.0000e+00 - val_loss: 7.6153
Epoch 2/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - categorical_accuracy: 0.0017 - loss: 7.6144
Epoch 2: val_loss did not improve from 7.61534
300/300 ━━━━━━━━━━━━━━━━━━━━ 48s 161ms/step - categorical_accuracy: 0.0017 - loss: 7.6144 - val_categorical_accuracy: 0.0000e+00 - val_loss: 7.6231
Epoch 3/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - categorical_accuracy: 8.3879e-04 - loss: 7.5975
Epoch 3: val_loss did not improve from 7.61534
300/300 ━━━━━━━━━━━━━━━━━━━━ 48s 159ms/step - categorical_accuracy: 8.3705e-04 - loss: 7.5975 - val_categorical_accuracy: 0.0000e+00 -